In [ ]:
import models
import numpy as np
import pandas as pd
from numpy import sqrt, exp, pi, power, tanh, vectorize
from scipy.integrate import odeint
from scipy.interpolate import make_interp_spline, interp1d
import matplotlib.pyplot as plt

In [ ]:
folder = ''

In [ ]:
### Read input file
input_file = pd.read_csv(folder+'input_irradiance_mel.csv', sep=";", decimal=",")
minE = 0
maxE = 1000
time_wake = 8.0 # 6.0
time_sleep = 20.0 #24.0

In [ ]:
### Create steps from input irradiance
input_file['hour'] = round(input_file.hours,0)
input_step = input_file[['irradiance_mel','hour']].groupby('hour').mean()
input_step.reset_index(inplace=True)

In [ ]:
### Smooth input irradiance with second degree polynomial
x = input_file.hours
a = np.polyfit(x, input_file.irradiance_mel, 2)
input_file['irradiance_mel_smooth'] = a[0] * power(x,2) + a[1] * x + a[2]

In [ ]:
output_file = pd.read_csv(folder+"output.csv", sep=";", decimal=",")
n = output_file.shape[0]
output_file['E_mel'] = output_file.I_mel*0.0013262 # convert to irradiance

In [ ]:
### Find nearest point in vector
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

### Return irradiance at time t
def irradiance(t):
    # add version here to choose between illuminance and irradiance
    t = t/3600
    t = t % 24
    if ((t < time_wake) or (t > time_sleep)): 
        E_emel = 0.036 # from the scientist: 27 lux
        I_emel = 27
    elif ((t < 9.0) or (t > 16.0)): 
        E_emel = 0.22 # from the scientist: 165 lux
        I_emel = 165
    else:
        ### original data
        idx = find_nearest(input_file.hours, t)
        E_emel = input_file.irradiance_mel[idx]
        I_emel = input_file.illuminance_mel[idx]
        ### smoothed data (second order poly)
        #E_emel = input_file.irradiance_mel_smooth[idx]
        ### aggregated by hour "step" data
        # idx = find_nearest(new_input.hour, t)
        # E_emel = input_step.irradiance_mel[idx]
        # E_emel = 0.22 # step
    return E_emel
    #return I_emel

irradiance_v = vectorize(irradiance)

In [ ]:
def forced_wake(t):
    # Testing with forced wake between t1 and t2
    if ((t/3600 % 24) >= time_wake and (t/3600 % 24) <= time_sleep):
        F_w = 1
    else:
        F_w = 0
    #F_w = 0
    return F_w

forced_wake_v = vectorize(forced_wake)

In [ ]:
#### Initial Conditions 
# [V_v, V_m, H, X, Y, P, Theta_L]

# y0 = [1.5, -15.0, 13.0, 0.04, -1.28, 0.0, 0.0] # initial values from draft
y0 = [ -4.55, -0.07, 13.29, -0.14, -1.07, 0.10, -5.00e-06] # proper initial values after experimentation

In [ ]:
### Execute the ODE model
t = np.linspace(0,72*60*60,n)
version_year = '2020'
(sol,temp) = odeint(models.model, y0, t, args = (irradiance, forced_wake, minE, maxE, version_year,), full_output = True)

### Store results
V_v = sol[:, 0]
V_m = sol[:, 1]
H = sol[:, 2]
X = sol[:, 3]
Y = sol[:, 4]
P = sol[:, 5]
Theta_L = sol[:, 6]
t_hours = t/3600

In [ ]:
### Plot ODEs
plt.figure(figsize=(10, 10))

plt.subplot(3,2,1)
plt.plot(t_hours, V_v, 'b', label='V_v(t)')
plt.plot(t_hours, V_m, 'g', label='V_m(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid(True)

plt.subplot(3,2,2)
plt.plot(t_hours, X, 'c', label='X(t)')
plt.plot(t_hours, Y, 'm', label='Y(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid(True)

plt.subplot(3,2,3)
plt.plot(t_hours, H, 'r', label='H(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid(True)

plt.subplot(3,2,4)
plt.plot(t_hours, P, 'r', label='P(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid(True)

plt.subplot(3,2,5)
plt.plot(t_hours, Theta_L, 'b', label='Theta_L(t)')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
### Compute all the internal functions
IE    = irradiance_v(t)
S     = models.state_v(V_m) 
alpha = models.photoreceptor_conversion_rate_v(IE, S, version_year)
Q_m   = models.mean_population_firing_rate_v(V_m)
Q_v   = models.mean_population_firing_rate_v(V_v)
C     = models.circadian_drive_v(X,Y)
D_v   = models.total_sleep_drive_v(H,C)
D_n   = models.nonphotic_drive_v(X, S)
D_p   = models.photic_drive_v(X, Y, P, alpha)
F_w   = forced_wake_v(t)
W     = models.wake_effort_v(Q_v, F_w)
Sigmoid = ( models.sigmoid_v(IE) - models.sigmoid_v(minE) ) / ( models.sigmoid_v(maxE) - models.sigmoid_v(minE) )

In [ ]:
### Plot internal variables
plt.figure(figsize=(10, 15))

plt.subplot(6,2,1)
plt.plot(t_hours, IE)
plt.xlabel('t')
plt.ylabel('I_mel(t)')
plt.grid(True)

plt.subplot(6,2,2)
plt.plot(t_hours, S)
plt.xlabel('t')
plt.ylabel('S(t)')
plt.grid(True)

plt.subplot(6,2,3)
plt.plot(t_hours, Sigmoid)
plt.xlabel('t')
plt.ylabel('Sigmoid(t)')
plt.grid(True)

plt.subplot(6,2,4)
plt.plot(t_hours, alpha)
plt.xlabel('t')
plt.ylabel('alpha(t)')
plt.grid(True)

plt.subplot(6,2,5)
plt.plot(t_hours, Q_m)
plt.xlabel('t')
plt.ylabel('Q_m(t)')
plt.grid(True)

plt.subplot(6,2,6)
plt.plot(t_hours, Q_v)
plt.xlabel('t')
plt.ylabel('Q_v(t)')
plt.grid(True)

plt.subplot(6,2,7)
plt.plot(t_hours, C)
plt.xlabel('t')
plt.ylabel('C(t)')
plt.grid(True)

plt.subplot(6,2,8)
plt.plot(t_hours, D_v)
plt.xlabel('t')
plt.ylabel('D_v(t)')
plt.grid(True)

plt.subplot(6,2,9)
plt.plot(t_hours, D_n)
plt.xlabel('t')
plt.ylabel('D_n(t)')
plt.grid(True)

plt.subplot(6,2,10)
plt.plot(t_hours, D_p)
plt.xlabel('t')
plt.ylabel('D_p(t)')
plt.grid(True)

plt.subplot(6,2,11)
plt.plot(t_hours, F_w)
plt.xlabel('t')
plt.ylabel('F_w(t)')
plt.grid(True)

plt.subplot(6,2,12)
plt.plot(t_hours, W)
plt.xlabel('t')
plt.ylabel('W(t)')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
AM = models.alertness_measure_v(C, H, Theta_L)

In [ ]:
### Plot AM-related variables
plt.figure(figsize=(5, 10))

plt.subplot(5,1,1)
plt.plot(t_hours, C)
plt.xlabel('t')
plt.ylabel('C(t)')
plt.grid(True)

plt.subplot(5,1,2)
plt.plot(t_hours, H)
plt.xlabel('t')
plt.ylabel('H(t)')
plt.grid(True)

plt.subplot(5,1,3)
plt.plot(t_hours, Theta_L)
plt.xlabel('t')
plt.ylabel('Theta_L(t)')
plt.grid(True)

plt.subplot(5,1,4)
plt.plot(t_hours, Sigmoid)
plt.xlabel('t')
plt.ylabel('Sigmoid(t)')
plt.grid(True)

plt.subplot(5,1,5)
plt.plot(output_file.time, output_file.KSS, 'darkorange')
plt.plot(t_hours, AM)
plt.xlabel('t')
plt.ylabel('KSS(t)')
plt.grid(True)

In [ ]:
### Plot AM and Irradiance
plt.figure(figsize=(5, 5))

plt.subplot(2,1,1)
plt.plot(output_file.time, output_file.E_mel, 'darkorange')
plt.plot(t_hours, IE)
plt.xlabel('t')
plt.ylabel('Irradiance(t)')
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(output_file.time, output_file.KSS, 'darkorange')
plt.plot(t_hours, AM)
plt.xlabel('t')
plt.ylabel('KSS(t)')
plt.grid(True)

In [ ]:
plt.plot(output_file.time, output_file.E_mel, 'darkorange', label='E_mel(t), Patricia')
plt.plot(t_hours, irradiance_v(t), 'b', label='E_mel(t), Victoria')
plt.legend(loc='best')
plt.xlabel('t')
plt.grid()
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('time (s)')
ax1.set_ylabel('KSS', color=color)
ax1.plot(t_hours, AM, color=color)
ax1.plot(output_file.time, output_file.KSS, color=color, linestyle='dashed')
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Irradiance', color=color)  # we already handled the x-label with ax1
ax2.plot(t_hours, irradiance_v(t), color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# find some initial conditions
idx = find_nearest(H, 13.3)
print("[ {}, {}, {}, {}, {}, {}, {}]".format(V_v[idx],V_m[idx],H[idx],X[idx],Y[idx],P[idx],Theta_L[idx]))


In [ ]:
# find some initial conditions
idx = find_nearest(t/3600, 48)
print("[ {}, {}, {}, {}, {}, {}, {}]".format(V_v[idx],V_m[idx],H[idx],X[idx],Y[idx],P[idx],Theta_L[idx]))

In [ ]:
x = np.linspace(0,10000,100)
y = 1/(1 + exp((0.05-x)/223.5) )
plt.plot(x, y)